In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [8]:
"""
DEFINIR VARIABLE PARA CONSEGUIR TODAS LAS URLS DE LA PÁGINA
"""
URLS = [] # Lista vacia que contenga todas las URLS de la página

URL_BASE = "https://www.fatsecret.com.ar/calor%C3%ADas-nutrici%C3%B3n/search?q=9+de+Oro&pg=" # Definimos la URL_BASE, la cual es la misma para todas las paginas

# Hacemos un loop que agregue a la URL_BASE el numero de página.

for num in range(0,271):

    URL = URL_BASE + str(num) 

    URLS.append(URL) # Agregamos a la lista vacia las URLS de todas las página

In [9]:
print(URLS[2])

https://www.fatsecret.com.ar/calor%C3%ADas-nutrici%C3%B3n/search?q=9+de+Oro&pg=2


In [10]:
"""
CREAMOS EL DATAFRAME QUE CONTENERA LOS PRODUCTOS
"""
Tabla_Nutricional_Productos = pd.DataFrame()

In [11]:
"""
LOOP PRINCIPAL PARA CONSEGUIR LA INFORMACIÓN NUTRICIONAL
"""

for url in URLS:
    
    page = requests.get(url)

    pagesoup = BeautifulSoup(page.content, "html.parser")

    tabla = pagesoup.find("td",class_="leftCell")

    Productos = tabla.find_all("a",class_="prominent")
    Producto = [caracter.text for caracter in Productos]
    
    Marcas = tabla.find_all("a",class_="brand")
    Marca = [caracter.text.replace("(","").replace(")","") if len(caracter.text) > 0 else None for caracter in Marcas]
            
    Cantidades = tabla.find_all("div",class_="smallText greyText greyLink")
    Cantidad = [caracter.text.replace("\r", "").replace("\n", "").replace("\t", "").split("-", 1)[0] for caracter in Cantidades]
            
    Calorias = tabla.find_all("div", class_="smallText greyText greyLink")
    Caloria = [caracter.text.replace("\r", "").replace("\n", "").replace("\t", "").split("-", 1)[-1].split("|")[0].strip().replace("Calorías:", "").strip() 
           for caracter in Calorias]
            
    Grasas = tabla.find_all("div", class_="smallText greyText greyLink")
    Grasa = [caracter.text.replace("\r", "").replace("\n", "").replace("\t", "").split("|", 1)[-1].split("|")[0].strip().replace("Grasa:", "").strip() 
         for caracter in Grasas]
            
    Carbohidratos = tabla.find_all("div", class_="smallText greyText greyLink")
    Carbohidrato = [caracter.text.replace("\r", "").replace("\n", "").replace("\t", "").split("|", 2)[-1].split("|")[0].strip().replace("Carbh:", "").strip()
                for caracter in Carbohidratos]
            
    Proteinas = tabla.find_all("div", class_="smallText greyText greyLink")
    Proteina = [caracter.text.replace("\r", "").replace("\n", "").replace("\t", "").split("|", 3)[-1].split("|")[0].strip().replace("Prot:", "").split("g", 1)[0] + "g".strip() 
            for caracter in Proteinas]
        
    if not Tabla_Nutricional_Productos.empty:
        columnas = ["Producto","Marca","Cantidad(g)","Caloria(kcal)","Grasa(g)","Carbohidrato(g)","Proteina(g)"]
        data_ = list(zip(Producto, Marca, Cantidad, Caloria, Grasa, Carbohidrato, Proteina))
        df = pd.DataFrame(columns=columnas,data=data_)
        Tabla_Nutricional_Productos = pd.concat([Tabla_Nutricional_Productos,df],ignore_index=True)
    else:
        columnas = ["Producto","Marca","Cantidad(g)","Caloria(kcal)","Grasa(g)","Carbohidrato(g)","Proteina(g)"]
        data_ = list(zip(Producto, Marca, Cantidad, Caloria, Grasa, Carbohidrato, Proteina))
        Tabla_Nutricional_Productos = pd.DataFrame(columns=columnas,data=data_)

In [12]:
Tabla_Nutricional_Productos["Proteina(g)"] = Tabla_Nutricional_Productos["Proteina(g)"].str.replace(",",".")
Tabla_Nutricional_Productos["Proteina(g)"] = Tabla_Nutricional_Productos["Proteina(g)"].str.replace("g","")
Tabla_Nutricional_Productos["Grasa(g)"] = Tabla_Nutricional_Productos["Grasa(g)"].str.replace(",",".")
Tabla_Nutricional_Productos["Grasa(g)"] = Tabla_Nutricional_Productos["Grasa(g)"].str.replace("g","")
Tabla_Nutricional_Productos["Carbohidrato(g)"] = Tabla_Nutricional_Productos["Carbohidrato(g)"].str.replace(",",".")
Tabla_Nutricional_Productos["Carbohidrato(g)"] = Tabla_Nutricional_Productos["Carbohidrato(g)"].str.replace("g","")
Tabla_Nutricional_Productos["Caloria(kcal)"] = Tabla_Nutricional_Productos["Caloria(kcal)"].str.replace("kcal","")
Tabla_Nutricional_Productos["Cantidad(g)"] = Tabla_Nutricional_Productos["Cantidad(g)"].str.extract(r'\((.*?)\)')
Tabla_Nutricional_Productos["Cantidad(g)"] = Tabla_Nutricional_Productos["Cantidad(g)"].str.replace("g","")



In [54]:
Tabla_Nutricional_Productos["Caloria(kcal)"] = Tabla_Nutricional_Productos["Caloria(kcal)"].astype("Int64")
Tabla_Nutricional_Productos["Grasa(g)"] = Tabla_Nutricional_Productos["Grasa(g)"].astype("Float64")
Tabla_Nutricional_Productos["Carbohidrato(g)"] = Tabla_Nutricional_Productos["Carbohidrato(g)"].astype("Float64")
Tabla_Nutricional_Productos["Proteina(g)"] = Tabla_Nutricional_Productos["Proteina(g)"].astype("Float64")